<a href="https://colab.research.google.com/github/asdfasdf0311/Optimal_location_selection/blob/main/00_MCLP_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치
!pip install pandas geopy pulp

# 라이브러리 임포트
import pandas as pd
from geopy.distance import geodesic
import pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 27.5 MB/s eta 0:00:00


## 데이터 준비 및 전처리

In [ ]:
# 예제 데이터 생성 (실제 데이터에 맞게 수정 필요)
data = {
    '버스정류장명': ['관설동 종점', '기업도시로', '의료기기종합지원센터', '누가베스트', '동부교사거리'],
    '정류장번호': [10023, 10025, 10027, 10028, 10029],
    '도로명주소': [None, None, None, None, None],
    '지번주소': [
        '강원특별자치도 원주시 관설동 309',
        '강원특별자치도 원주시 지정면 가곡리 1482',
        '강원특별자치도 원주시 지정면 가곡리 1322',
        '강원특별자치도 원주시 지정면 가곡리 1320',
        '강원특별자치도 원주시 단구동 1506'
    ],
    '위도': [37.300581, 37.372115, 37.370856, 37.370469, 37.318103],
    '경도': [127.982576, 127.870087, 127.873331, 127.873924, 127.969723],
    '기준일': ['2024-03-31'] * 5,
    '인원': [702, 31, 26, 51, 204],
    # 추가적인 기준 (예제 값)
    'floating_population': [500, 300, 200, 400, 350],
    'tourist_visit_ratio': [0.2, 0.1, 0.05, 0.15, 0.25],
    'bus_ratio': [0.3, 0.2, 0.1, 0.25, 0.35]
}

df2 = pd.DataFrame(data)


## 거리 계산 및 커버리지 설정

In [ ]:
# 커버리지 반경 설정 (킬로미터 단위)
coverage_radius = 0.4  # 400m

# 모든 정류장 간 거리 계산 및 커버리지 설정
# 각 정류장이 다른 정류장을 커버할 수 있는지 확인
coverage = {}
for i, row_i in df2.iterrows():
    coverage[i] = []
    loc_i = (row_i['위도'], row_i['경도'])
    for j, row_j in df2.iterrows():
        loc_j = (row_j['위도'], row_j['경도'])
        distance = geodesic(loc_i, loc_j).kilometers
        if distance <= coverage_radius:
            coverage[i].append(j)


## 가중치 적용 및 점수 계산

In [ ]:
# 가중치 설정
weights = {
    'population_density': 0.5993,
    'floating_population': 0.1774,
    'tourist_visit_ratio': 0.1031,
    'bus_ratio': 0.1203
}

# 각 정류장의 점수 계산
df2['score'] = (
    df2['인원'] * weights['population_density'] +
    df2['floating_population'] * weights['floating_population'] +
    df2['tourist_visit_ratio'] * weights['tourist_visit_ratio'] +
    df2['bus_ratio'] * weights['bus_ratio']
)


## MCLP 모델 설정 및 해결

In [ ]:
# 최적화 모델 생성
model = pulp.LpProblem("MCLP", pulp.LpMaximize)

# 의사결정 변수: 각 정류장이 시설으로 선택되는지 여부
x = pulp.LpVariable.dicts('select', df2.index, cat='Binary')

# 각 정류장이 커버되는지 여부
y = pulp.LpVariable.dicts('cover', df2.index, cat='Binary')

# 목표 함수: 커버되는 정류장의 점수 합 최대화
model += pulp.lpSum([df2.loc[j, 'score'] * y[j] for j in df2.index])

# 제약 조건 1: 선택할 시설의 수는 최대 k개
k = 2  # 예: 최대 2개의 시설 선택
model += pulp.lpSum([x[i] for i in df2.index]) <= k

# 제약 조건 2: 각 정류장은 최소 하나의 커버리지를 가져야 함
for j in df2.index:
    model += y[j] <= pulp.lpSum([x[i] for i in coverage[i] if j in coverage[i]])
    # Optional: y[j] can only be 1 if at least one covering facility is selected
    # model += y[j] <= pulp.lpSum([x[i] for i in coverage[j]])

# 모델 해결
model.solve()

# 결과 출력
selected = [i for i in df2.index if pulp.value(x[i]) == 1]
print("선택된 버스 정류장:")
print(df2.loc[selected, ['버스정류장명', '지번주소']])


선택된 버스 정류장:
   버스정류장명                  지번주소
4  동부교사거리  강원특별자치도 원주시 단구동 1506


## 결과 시각화 및 해석

In [ ]:
import folium

# 지도 초기화 (원주시 중심으로 설정)
center_location = [df2['위도'].mean(), df2['경도'].mean()]
m = folium.Map(location=center_location, zoom_start=12)

# 모든 정류장 표시
for idx, row in df2.iterrows():
    folium.CircleMarker(
        location=(row['위도'], row['경도']),
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=row['버스정류장명']
    ).add_to(m)

# 선택된 정류장 강조 표시
for idx in selected:
    row = df2.loc[idx]
    folium.Marker(
        location=(row['위도'], row['경도']),
        popup=row['버스정류장명'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# 지도 표시
m
